# Apresentação

**Resumo:**

O estudo aqui apresentado tem o objetivo de apresentar conceitos de séries temporais, ferramentas úteis que podem ser usadas no dia-a-dia de pessoas interessadas em ciência de dados, e apontar cuidados que novos desenvolvedores devem ter quando iniciam o aprendizado neste tema. Isso é feito considerando dados relacionados ao Covid-19, tema importante no contexto atual, e que tem mobilizado muita força de trabalho de desenvolvedores e cientistas no mundo todo.

**Não é objetivo nosso desenvolver e otimizar modelos de predição**. Os modelos aqui mostrados e parametrizados adotam abordagens de simples entendimento para novos estudantes.

---

**Autores:**

*   [Carlos Maurício Seródio Figueiredo](https://www.linkedin.com/in/cmsfigueiredo/): É professor adjunto da Escola Superior de Tecnologia da Universidade do Estado do Amazonas (EST/UEA) e professor colaborador do Programa de Pós-graduação em Informática da Universidade Federal do Amazonas, é co-fundador do Lab. de Sistemas Inteligentes da UEA e pesquisador no Samsung Ocean Manaus.
*   [Alice Adativa Ferreira Menezes](https://www.linkedin.com/in/alice-adativa/): É doutoranda na Universidade Federal do Amazonas (UFAM) e  pesquisadora no Samsung Ocean.

---

**Ressalvas:** 

Chamamos a atenção para que, embora modelos e resultados sejam confrontados com a realidade, por vezes de forma bem aproximada, as análises dos dados apresentam um alto grau de incerteza devido a vários fatores:


*   Natureza imprecisa dos dados utilizados, podendo esses serem incompletos ou conterem erros;

*   Natureza imprecisa das metodologias de coleta de casos e óbitos por Covid-19, sabidamente contendo subnotificações e testes imprecisos;

*   Diferentes países, estados e cidades apresentam diferenças em infra-estrutura e socio-culturais enormes, adotando diferentes abordagens perante a pandemia do Covid-19 e, consequentemente, apresentando desempenhos muito variáveis;

*   A pandemia do Covid-19 supreendeu toda a comunidade científica internacional pelas suas características distintas das demais, levando a uma situação totalmente nova, e cuja realidade só será entendida com maior precisão depois que a mesma acabar, com posse de muitos dados e estudos.

---

**Atenção!!!!**

Previsão em Ciência de Dados não é saber o que vai acontecer amanhã. Previsão em Ciência de Dados é analisar padrões temporais e, com isso, estimar melhor as possibilidades de ocorrências futuras.

A resposta de modelos preditivos não é "no dia tal, o número de mortes será tanto", e sim "considerando que o histórico observado é representativo, que o compartamento se manterá, estimamos que no dia tal o número de mortes pode chagar a tanto".

**"Essencialmente, todos os modelos estão errados, mas alguns são úteis" (George Box).**


Obrigado por olhar nosso trabalho. Por favor, não hesite em deixar contribuições nos comentários.

**Observações realizadas com base nos dados até 18/05/2020.**







# Introdução

Uma série temporal é uma sequência de realizações (observações) de uma variável ao longo do tempo (WOOLDRIDGE, Jeffrey M. Introductory Econometrics: a Modern Approach. 2000, South-Western College Publishing, a division of Thomson Learning). Os dados de número de casos e óbitos, divulgados de forma oficial e diariamente, são exemplos de séries temporais.

Uma característica muito importante deste tipo de dado é que as observações vizinhas são dependentes, ou seja, dados passados se relacionam com os próximos, e o interesse em analisar e modelar essa dependência pode nos ajudar a entender seu comportamento histórico, identificando seus padrões e, com isso, estimar melhor o futuro. Por isso, muitas vezes encontramos o termo Previsão de Séries Temporais. E dessa forma, o estudo de séries temporais é importante em diferentes áreas do conhecimento, tais como  estatística, econometria, matemática aplicada e processamento de sinais.

Para saber mais:


*   Veja nossos cursos gratuitos no [Samsung Ocean](www.oceanbrasil.com.br).
*   [Conceitos de séries temporais](https://pt.wikipedia.org/wiki/S%C3%A9rie_temporal).
*   Robert H. Shumway. Time series analysis and its applications. Springer, 2000.
*   Aileen Nielsen. Practical Time Series Analysis. O’Reilly, 2019.


Nesse estudo, demonstraremos aspectos práticos com Python na criação de modelos de séries temporais e usaremos as seguintes bibliotecas:


*  Regressão Linear e Polinomial:  [Scikit-learn](https://scikit-learn.org/stable/).
*  SARIMA: [statsmodel](https://www.statsmodels.org/) e [auto-arima](https://www.alkaline-ml.com/pmdarima).
*  [Facebooks Prophet](https://facebook.github.io/prophet/).



# Coleta e Preparação

## Coleta de Dados

In [ ]:
import pandas as pd


#Dados internacionais
c=pd.read_csv('../input/cpia-local-de-datahubs-covid19-dataset/time-series-19-covid-combined.csv')
df_sumario_w = c.groupby(['Date','Country/Region'],as_index = False)['Confirmed','Deaths'].sum().pivot('Date','Country/Region').fillna(0)
df_conf_w = df_sumario_w["Confirmed"]
df_deaths_w = df_sumario_w["Deaths"]

#Dados População Internacionais
popw = pd.read_excel('../input/cpia-local-de-covid19-ecdc-dataset/COVID-19-geographic-disbtribution-world.xlsx')
popw["countriesAndTerritories"] = popw["countriesAndTerritories"].str.replace("United_States_of_America", "US", case = False)
popw['countriesAndTerritories'] = popw['countriesAndTerritories'].str.replace("_"," ")
pop = popw.groupby('countriesAndTerritories', as_index=False).max()[['countriesAndTerritories','popData2018']]
pop.rename(columns={'countriesAndTerritories':'local', 'popData2018':'pop'}, inplace=True);

# Dados Nacionais
df = pd.read_csv('../input/corona-virus-brazil/brazil_covid19.csv')
dfe = df.drop(['region'], axis=1) #
dfe_sumario = dfe.groupby(['date','state'],as_index = False)['cases','deaths'].sum().pivot('date','state').fillna(0)
dfe_conf = dfe_sumario["cases"]
dfe_deaths = dfe_sumario["deaths"]

## Funções Úteis

#### acum2pd

In [ ]:
# Por dia
def acum2pd(df_conf):
    
    df_conf_pd = df_conf.copy()

    for col in df_conf.columns:
        for i,x in enumerate(df_conf_pd[col]):
            if i != 0:
                pdia = df_conf[col][i]-df_conf[col][i-1]
                if pdia < 0:
                    df_conf_pd[col][i] = 0
                    df_conf[col][i] = df_conf[col][i-1]
                else:
                    df_conf_pd[col][i] = pdia
    return df_conf_pd

### sigm_predict

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def sigm_predict(y, future_days, boundary):
    def avg_err(pcov):
        return np.round(np.sqrt(np.diag(pcov)).mean(), 2)
    # function to be minimized
    def f_sigmoid(x, a, b, c):
        # a = sigmoid midpoint
        # b = curve steepness (logistic growth)
        # c = max value
        return (c / (1 + np.exp(-b*(x-a))))
  
    x = np.arange(len(y))
    
    # fitting the data on the logistic function
    #boundary = ([-np.inf, -np.inf, -np.inf],[np.inf, np.inf, np.inf])
    #boundary = ([0., 0.001, y.max()],[90., 2.5, 100*y.max()])
    popt_sig, pcov_sig = curve_fit(f_sigmoid, x[:-1], y[:-1], method='trf',bounds=boundary)#,sigma = np.linspace(0.5, 0.05, len(y)),absolute_sigma=True)#dogbox, sigma = np.linspace(0.5, 0.05, len(y)),absolute_sigma=False
    peakday = len(y) + int(popt_sig[0])
     
    x_m = np.arange(len(y)+future_days)
    y_m = f_sigmoid(x_m, *popt_sig)    
    
    return x_m, y_m, avg_err(pcov_sig), popt_sig

# Entendendo os dados

Quando analisamos dados do Covid, como neste dataset e nos complementos aqui utilizados, temos acessos a dados gerais de divulgação, como quantidade de casos acumulados, quantidade de óbitos acumulados, nos diferentes países e em uma periodicidade diária. É um nídido exemplo de uma série temporal. 

O objetivo desses dados é mostrar a evolução da pandemia em um determinado local. Assim, as pessoas podem entender os estágio de evolução da mesma, se a mesma está crescendo ou não, se as medidas de isolamento social estão surtindo efeito etc.

Abaixo exemplos comparativos de alguns países e de alguns estados brasileiros.

Na seção de Coleta e Preparação de dados, abrimos o dataset e fizemos algumas minipulações para resumí-los dessa forma: Cada país a quantidade de casos acumulados por dia (df_conf_w) e mortes por dia (df_deaths_w).

In [ ]:
df_conf_w.tail()

In [ ]:
df_deaths_w.tail()

Melhor plotar alguns países pra ver melhor.

In [ ]:
plt.figure(figsize=(16,8))
ax = plt.subplot(1,2,1)
ax.set_title('Casos Acumulados', fontsize=18, loc='left')
plt.plot(df_conf_w['Germany'], label='Germany')
plt.plot(df_conf_w['Italy'], label='Italy')
plt.plot(df_conf_w['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();
ax = plt.subplot(1,2,2)
ax.set_title('Óbitos Acumulados', fontsize=18, loc='left')
plt.plot(df_deaths_w['Germany'], label='Germany')
plt.plot(df_deaths_w['Italy'], label='Italy')
plt.plot(df_deaths_w['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Óbitos")
plt.legend();

Uma forma alternativa de visualizar a evolução dos casos, é na visão por dia, que também preparamos em df_conf_pd e df_deaths_pd. Essa visão é mais próxima das pessoas, porque logo enxergam quantos estão sendo contaminados ou vindo a óbito por dia.

In [ ]:
df_conf_pd = acum2pd(df_conf_w)
df_deaths_pd = acum2pd(df_deaths_w)

In [ ]:
plt.figure(figsize=(16,8))
ax = plt.subplot(1,2,1)
ax.set_title('Casos por Dia', fontsize=18, loc='left')
plt.plot(df_conf_pd['Germany'], label='Germany')
plt.plot(df_conf_pd['Italy'], label='Italy')
plt.plot(df_conf_pd['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();
ax = plt.subplot(1,2,2)
ax.set_title('Óbitos por Dia', fontsize=18, loc='left')
plt.plot(df_deaths_pd['Germany'], label='Germany')
plt.plot(df_deaths_pd['Italy'], label='Italy')
plt.plot(df_deaths_pd['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Óbitos")
plt.legend();

Da mesma forma, para alguns estados do Brasil.

In [ ]:
dfe_conf_pd = acum2pd(dfe_conf)
dfe_deaths_pd = acum2pd(dfe_deaths)

In [ ]:
## Graficos Estados do Brasil
plt.figure(figsize=(16,8))
ax = plt.subplot(1,2,1)
ax.set_title('Casos por Dia', fontsize=18, loc='left')
plt.plot(dfe_conf_pd['São Paulo'], label='SP')
plt.plot(dfe_conf_pd['Amazonas'], label='AM')
plt.plot(dfe_conf_pd['Santa Catarina'], label='SC')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();
ax = plt.subplot(1,2,2)
ax.set_title('Óbitos por Dia', fontsize=18, loc='left')
plt.plot(dfe_deaths_pd['São Paulo'], label='SP')
plt.plot(dfe_deaths_pd['Amazonas'], label='AM')
plt.plot(dfe_deaths_pd['Santa Catarina'], label='SC')
plt.xlabel("Dia")
plt.ylabel("Óbitos")
plt.legend();

O interessante da visão diária é que podemos observar que a evolução dos dados não é tão bem comportada, apresentando variações devido a ruídos, tais como a própria natureza estocástica de casos e mortes, limitações em testes ou no reporte deles etc. Ainda é interessante ver em casos de países em estágios mais avançados da epidemia, que o a evolução se dá com um crescimento do número de casos por dia, chega num pico (o famoso pico) e depois começa a decrescer. As taxas de crescimento e descrescimento podem ser devidas a evolução natural de uma epidemia, cujos modelos oficiais mostram que a mesma segue um formato da função sigmoid (ou logística) quando acumulados, que correspondem a um sino quando por dia (ver curvas da Alemanha), ou ainda a fatores como ações de isolamento social etc.

No caso da Alemanha, observa-se nitidamente de que há indícios de que um pico de casos ocorreu 02/04 e o de óbitos em 15/04. (No momento desse estudo, não é possível afirmar ainda se ele é devido ao ciclo natural da epidemia ou se é um controle realizado com medidas de contenção. Se for devido ao último fator, sua curva deve apresentar calda longa ou novas ondas).
No caso do Brazil, vemos que o mesmo começou mais atrasado na pandemia, e não há sinais de pico nos gráficos (curva acumulada ainda não se parece uma sigmoid e casos por dia com tendência de crescimento).

Já olhando os estados brasileiros, vemos que como o país é continental, temos também muitas diferenças entre eles. Ou estão em diferentes momentos da epidemia, ou fatores regionais e socio-culturais podem ajudar um estado a conter melhor a epidemia (não vamos entrar nessas questões aqui).

Observar o pico de uma epidemia é um aspecto importante, pois nos possibilita enxergar quando começaremos a regredir no número de casos e óbitos e quando esta deve cessar.

Então a questão é:

**Como estimar o pico?**

Vamos chegar lá. Mas vamos ver algumas coisinhas primeiro.


# Como será o amanhã?

"... 
Responda quem puder
O que irá me acontecer?
O meu destino será
Como Deus quiser
Como será?" (João Sérgio, GRES União da Ilha do Gov).

No caso em estudo, monitorando os casos acumulados do vírus, prever o amanhã é interessante mas de pouco impacto. Do ponto de vista prático, espera-se que uma epidemia não mude de tendência tão bruscamente de um dia para outro. Então qualquer previsão inocente, como ver a diferença dos dias anteriores e acrescentar ao último dia, pode dar resultados próximos. E se errar, será por pouco, se comparado proporcionalmente ao número acumulado de casos. 

Vamos pegar um exemplo bem rápido, para o Brasil, vamos plotar as duas curvas. Quase não notamos a diferença nos casos acumulados, embora ela seja mais evidente quando olhamos por dia.

In [ ]:
br = dfe_conf['Amazonas'][:75]
br1 = br.copy()
br1[-1] = br1[-2] + br1[-2] - br1[-3]

In [ ]:
plt.figure(figsize=(16,8))
ax = plt.subplot(1,2,1)
ax.set_title('Casos Acumulados', fontsize=18, loc='left')
plt.plot(br, 'b', label='AM')
plt.plot(br1, 'r', label='Prev')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();
ax = plt.subplot(1,2,2)
ax.set_title('Casos por Dia', fontsize=18, loc='left')
plt.plot(np.diff(br), 'b', label='AM')
plt.plot(np.diff(br1), 'r', label='Prev')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Então, nesse estudo, vamos considerar um desafio um pouco maior, a possibilidade de prever mais dias à frente. Do ponto de vista didático, podemos observar melhor questões como tendência e sazonalidade que podem ocorrer nos dados. E considere, inclusive, que prever mais dias à frente seria mais útil para cientistas e gestores, pois poderiam ter tempo de tomar algumas medidas de contorno, tais como preparar leitos ou remédios.

## Abordagens de Previsões Autoregressivas

Uma abordagem comum, quando temos dados históricos de algum fenômeno e queremos prever o futuro, e olhar o padrão contido nesses dados e tentar projetá-lo para a frente.

A abordagem mais simples é a **Regressão Linear Simples**, que visa encontrar uma reta que melhor representa os dados. Então, de posse dessa reta, podemos projetar novos pontos.

No caso do Brasil, vamos usar os dados iniciais, excluindo os 5 dias finais, e ver se poderíamos tê-los estimado adequadamente. 

### Regressão Linear Simples

In [ ]:
#Lib scikit-learn
from sklearn.linear_model import LinearRegression
import numpy as np

#A partir do 100o caso
casos_br = df_conf_pd['Brazil'][df_conf_pd[df_conf_pd['Brazil'] > 100].index[0]:].values
#Dividindo os dados do passado e os a serem previstos
casos_br_treino = casos_br[:-5]
casos_br_teste = casos_br[-5:]
#Instanciando o modelo
model = LinearRegression()
#Botando no formato do fit
X = np.expand_dims(np.linspace(0,len(casos_br_treino), len(casos_br_treino)), axis=1)
y = np.expand_dims(casos_br_treino, axis=1)
#Fazendo o treino
model.fit(X,y)
#Prevendo com dias à frente
casos_br_prev = model.predict(np.expand_dims(np.linspace(0, len(casos_br_treino)+5,len(casos_br)), axis=1))

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.linspace(0,len(casos_br_treino), len(casos_br_treino)), casos_br_treino, 'b', label='Passado')
plt.plot(np.linspace(len(casos_br_treino), len(casos_br_treino)+5, 5), casos_br_teste, 'g', label = 'Futuro')
plt.plot(np.linspace(0, len(casos_br_prev), len(casos_br_prev)), casos_br_prev, 'r', label = 'Previsão')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Como podemos observar, essa opção não é boa, pois nitidamente o comportamento da evolução dos casos não é uma reta. Nesse caso, um obsevador mais cuidadoso poderia imaginar que outra função seria mais apropriada. Por exemplo, fazer um fit de uma função **exponencial**, como parece o trecho do gráfico, ou uma função **polinomial** que poderia se adequar melhor ao padrão.

Nesse caso, vamos tentar uma regressão de uma função polinomial que é capaz de adequar uma variedade grande de curvas.

### Regressão de uma função Polinomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 

# Instanciando Modelo  
poly = PolynomialFeatures(degree = 4) 
X_poly = poly.fit_transform(X) 
# Treino  
poly.fit(X_poly, y) 
lin2 = LinearRegression() 
lin2.fit(X_poly, y)
# Predição
casos_br_prev_poli = lin2.predict(poly.fit_transform(np.expand_dims(np.linspace(0, len(casos_br_treino)+5,len(casos_br)), axis=1)))

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.linspace(0,len(casos_br_treino), len(casos_br_treino)), casos_br_treino, 'b', label='Passado')
plt.plot(np.linspace(len(casos_br_treino), len(casos_br_treino)+5, 5), casos_br_teste, 'g', label = 'Futuro')
plt.plot(np.linspace(0, len(casos_br_prev), len(casos_br_prev)), casos_br_prev_poli, 'r', label = 'Previsão')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Parece muito melhor. E a regressão de uma função polinomial de 4a ordem modelou o comportamento exponencial da curva de crescimento de casos. No intervalo mostrado, a tendência está muito bem representada.

Mas será que não poderíamos melhorar a previsão com uso de modelos mais completos, que capturam mais características da série temporal? Por exemplo, a curva apresenta algumas oscilações que parecem ter uma sazonalidade.

Para isso, vamos analisar características temporais dos dados. Uma forma clássica de análise de séries temporais é decompô-la em tendência, sazonalidade e ruídos. Para tentar visualizar sazonalidade, vamos primeiro fazer uma análise de autocorrelação.

### Análise Temporal

Para tentar visualizar sazonalidade, vamos primeiro fazer uma análise de autocorrelação.

#### Autocorrelação

In [ ]:
from pandas.plotting import autocorrelation_plot

ax = plt.figure(figsize=(12,6))
ax.suptitle("Autocorrelação")
autocorrelation_plot(casos_br_treino)
ax=ax

Como podemos observar, parece haver uma sazonalidade forte nos dados considerando um lag pouco menor que 10 amostras dos dados, ou seja, 10 dias. Se observarmos bem os dados de casos por dia, podemos ver que eles costumam ter uma redução em fins de semana. A sazonalidade é de 7 dias, e fica evidente no gráfico de autocorrelação.

#### Decomposição

Vamos, então, decompor os dados considerando uma frequência de 7 dias.

Assim, podemos observar nitidamente que há uma forte tendência exponencial (Trend) e um padrão sazonal de 7 dias nos dados, além de alguns ruídos espalhados pelo universo de dados.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams

rcParams['figure.figsize'] = 11, 9

dec = seasonal_decompose(casos_br_treino, freq=7)
dec.plot();


### Modelo SARIMA

Se observamos tendência e sazonalidade, um modelo melhor de previsão de séries temporais é o SARIMA, que considera a composição de uma modelo Sazonal (S), Autoregressivo (AR), Integral (I) e de média móvel(MA). 

Para a parametrização, vamos logo apelar para as possibilidades de parâmetros de modelagem fazendo um busca com o **Auto ARIMA**.

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima

# Instancia modelo já com os melhores parâmetros buscados automaticamente
m_sarima = auto_arima(casos_br_treino,
                     start_p=0, start_q=0, max_p=10, max_q=10,
                     d=1, m=7, seasonal = True,
                     D=1, start_Q=1, start_P=1, max_P=4, max_Q=4,
                     trace=True,
                     error_action='ignore', suppress_warnings=True,
                     stepwise=False)

In [ ]:
#Treino
m_sarima.fit(casos_br_treino)
#Predição
fitted = m_sarima.predict_in_sample(start=1, end=len(casos_br_treino))
casos_br_prev_sarima = m_sarima.predict(n_periods=len(casos_br_teste))

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.linspace(0,len(casos_br_treino), len(casos_br_treino)), casos_br_treino, 'b', label='Passado')
plt.plot(np.linspace(0,len(casos_br_treino), len(casos_br_treino)), fitted, 'r')
plt.plot(np.linspace(len(casos_br_treino), len(casos_br_treino)+5, 5), casos_br_teste, 'g', label='Futuro')
plt.plot(np.linspace(len(casos_br_treino), len(casos_br_treino)+5, 5), casos_br_prev_sarima, 'r', label='Previsão')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend()

Parece que capturou melhor tendência e sazonalidade.
Mas então, podemos seguir assim, período após período, e chegar até o fim da epidemia?

Não, não, pequeno Padawan. E se a tendência mudar? Veja bem, estamos olhando o passado para prever o futuro. Então o passado mostra uma tendência que pode mudar, logo, previsões após essas mudanças divergiriam da realidade.

Como exemplo, vamos pegar novamente os dados da Alemanha, antes do pico, e observar como seriam as projetos dos modelos estimados.

### Projeções na fase de crescimento

Dados da Alemanha, por dia, a partir do 10o caso, podemos observar que o pico está por volta do dia 30. Vamos projetar 5 dias a partir do dia 25 e verificar se os modelos capturam a característica da epidemia.

In [ ]:
#A partir do 10o caso
casos_al = df_conf_pd['Germany'][df_conf_pd[df_conf_pd['Germany'] > 10].index[0]:].values
plt.plot(casos_al);

In [ ]:
#Dividindo os dados do passado e os a serem previstos
casos_al_treino = casos_al[:25]
casos_al_teste = casos_al[25:]

num_proj = 10
# Regressão Linear
model = LinearRegression()
X = np.expand_dims(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), axis=1)
y = np.expand_dims(casos_al_treino, axis=1)
model.fit(X,y)
casos_al_prev = model.predict(np.expand_dims(np.linspace(0,len(casos_al_treino)+num_proj,len(casos_al_treino)+num_proj), axis=1))

#Regressão Polinimial
poly = PolynomialFeatures(degree = 3) 
X_poly = poly.fit_transform(X) 
poly.fit(X_poly, y) 
lin2 = LinearRegression() 
lin2.fit(X_poly, y)
casos_al_prev_poli = lin2.predict(poly.fit_transform(np.expand_dims(np.linspace(0, len(casos_al_treino)+num_proj,len(casos_al_treino)+num_proj), axis=1)))

#SARIMA
m_sarima = auto_arima(casos_al_treino,
                     start_p=0, start_q=0, max_p=10, max_q=10,
                     d=1, m=7, seasonal = True,
                     D=1, start_Q=1, start_P=1, max_P=4, max_Q=4,
                     trace=True,
                     error_action='ignore', suppress_warnings=True,
                     stepwise=False)
safit = m_sarima.fit(casos_al_treino)
fitted = m_sarima.predict_in_sample(start=1, end=len(casos_al_treino))
casos_al_prev_sarima = m_sarima.predict(n_periods=num_proj)

In [ ]:
plt.figure(figsize=(10,4))
#plt.plot(np.linspace(0,len(casos_al), len(casos_al)), casos_al, 'b', label='Real')
plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), casos_al_treino, 'b', label='Passado')
plt.plot(np.linspace(len(casos_al_treino), len(casos_al), len(casos_al)-len(casos_al_treino)), casos_al[len(casos_al_treino):], 'g', label='Futuro')
plt.plot(np.linspace(0,len(casos_al_treino)+num_proj, len(casos_al_treino)+num_proj), casos_al_prev, 'r', label='Linear')
plt.plot(np.linspace(0,len(casos_al_treino)+num_proj, len(casos_al_treino)+num_proj), casos_al_prev_poli, 'y', label='Poli')
plt.plot(np.linspace(len(casos_al_treino), len(casos_al_treino)+num_proj, num_proj), casos_al_prev_sarima[:], 'c', label='SARIMA')
plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), fitted, 'c')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Olha que interessante!!! O modelo linear ficou com previsões aquém da realidade, pois foi muito afetado pelo aprendizado de uma fase mais suave da curva. O modelo polinomial conseguiu capturar melhor a tendência. E o SARIMA refletiu muito bem os dados futuros, inclusive com a sazonalidade embutida!

**Problema resolvido?**

Vamos ver para valores mais próximos do pico. Dessa vez, vamos assumir que já estamos no dia 30.

### Revendo para Projeções Próxima ao Pico

In [ ]:
#Dividindo os dados do passado e os a serem previstos
casos_al_treino = casos_al[:30]
casos_al_teste = casos_al[30:]

num_proj = 10
# Regressão Linear
model = LinearRegression()
X = np.expand_dims(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), axis=1)
y = np.expand_dims(casos_al_treino, axis=1)
model.fit(X,y)
casos_al_prev = model.predict(np.expand_dims(np.linspace(0,len(casos_al_treino)+num_proj,len(casos_al_treino)+num_proj), axis=1))

#Regressão Polinimial
poly = PolynomialFeatures(degree = 3) 
X_poly = poly.fit_transform(X) 
poly.fit(X_poly, y) 
lin2 = LinearRegression() 
lin2.fit(X_poly, y)
casos_al_prev_poli = lin2.predict(poly.fit_transform(np.expand_dims(np.linspace(0, len(casos_al_treino)+num_proj,len(casos_al_treino)+num_proj), axis=1)))

#SARIMA
m_sarima = auto_arima(casos_al_treino,
                     start_p=0, start_q=0, max_p=10, max_q=10,
                     d=1, m=7, seasonal = True,
                     D=1, start_Q=1, start_P=1, max_P=4, max_Q=4,
                     trace=True,
                     error_action='ignore', suppress_warnings=True,
                     stepwise=False)
safit = m_sarima.fit(casos_al_treino)
fitted = m_sarima.predict_in_sample(start=1, end=len(casos_al_treino))
casos_al_prev_sarima = m_sarima.predict(n_periods=num_proj)

In [ ]:
plt.figure(figsize=(10,4))
#plt.plot(np.linspace(0,len(casos_al), len(casos_al)), casos_al, 'b', label='Real')
plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), casos_al_treino, 'b', label='Passado')
plt.plot(np.linspace(len(casos_al_treino), len(casos_al), len(casos_al)-len(casos_al_treino)), casos_al[len(casos_al_treino):], 'g', label='Futuro')
plt.plot(np.linspace(0,len(casos_al_treino)+num_proj, len(casos_al_treino)+num_proj), casos_al_prev, 'r', label='Linear')
plt.plot(np.linspace(0,len(casos_al_treino)+num_proj, len(casos_al_treino)+num_proj), casos_al_prev_poli, 'y', label='Poli')
plt.plot(np.linspace(len(casos_al_treino), len(casos_al_treino)+num_proj, num_proj), casos_al_prev_sarima[:], 'c', label='SARIMA')
plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), fitted, 'c')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend()

Vixiiii! A regressão linear descolou e continua subindo. A polinomial, então, foi-se embora! O modelo SARIMA ainda tentou representar a sazonalidade, mas manteve a tendência de subida. Por que? Porque aprenderam pelos dados de passado que tinha a tendência de crescimento, e os dados seguintes mostraram a inversão da mesma.

Ou seja, esses modelos só servem para **representar futuros próximos**, sem mudanças de tendências. E o modelo da epidemia tem que inverter a tendência, pois a população não é infinita. Uma hora, quando muitos estiverem contaminados, teremos menos pessoas a contaminar. O número de casos por dia tem que subir, chegar no pico e descer.

Lembra de alguma coisa? Huff explicou em 1954: 

**Não extrapole nas extrapolações!**


In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://statsland.files.wordpress.com/2012/08/extrapolating.jpg")

Resumindo, muita gente prevê os próximos dias com esses modelos, o que é útil em fases bem comportadas da curva, ou seja, sem grandes mudanças de tendência.

Mas como nota-se, eles não serão razoáveis para descobrir o pico nem modelar a pandemia como um todo.

E agora, José? Uma pergunta comum em tempos de pandemia é: 

**Quando será o pico?**

O motivo é o desejo de saber se a situação ainda vai piorar ou vai começar a melhorar. Responder essa pergunta é o esfoço de muitos cientistas ao redor do mundo.

## Conhecimento de Domínio


Bom, temos que ter um pouco de conhecimento de domínio para saber que uma epidemia não evolui eternamente. Ela tem uma fase de expansão agressiva, similar a exponencial, depois começa a estabilizar, conforme a população vai ficando imune, e então começa a decrescer até esvair-se. A população não é infinita, logo a epidemia tem que ter um limite de crescimento.

Epidemias costumam ser modeladas por funções sigmoids, quando olhamos casos acumulados, que expressam exatamente esse comportamento. Se olharmos por dia, a função sigmoid se assemelha a um sino.

Vejamos alguns exemplos. Aqui plotamos os casos acumulados da Alemanha, Itália, China e Brasil. Vemos que todas as curvas acumuladas aprensentam o comportamento sigmoid. Mas não é à toa. Modelos científicos de epidemias, como o modelo SIR, são baseados nessa curva.

No caso do Brasil, vemos que ainda não chegamos na mudança de tendência, ou seja, na amortização dos casos acumulados. De outra forma, não vemos o pico.

**Mas onde esse trem tá????**

In [ ]:
plt.figure(figsize=(16,8))
ax = plt.subplot(1,2,1)
ax.set_title('Casos Acumulados', fontsize=18, loc='left')
plt.plot(df_conf_w['Germany'], label='Germany')
plt.plot(df_conf_w['Italy'], label='Italy')
plt.plot(df_conf_w['China'], label='China')
plt.plot(df_conf_w['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();
ax = plt.subplot(1,2,2)
ax.set_title('Casos por Dia', fontsize=18, loc='left')
plt.plot(df_conf_pd['Germany'], label='Germany')
plt.plot(df_conf_pd['Italy'], label='Italy')
plt.plot(df_conf_pd['China'], label='China')
plt.plot(df_conf_pd['Brazil'], label='Brazil')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Então vamos fazer o seguinte: como melhor modelo de previsão, vamos fazer um fit de uma função logística (sigmoid) nos dados, até então, e ver se a projeção dos casos futuros reflete melhor a realidade e, com isso, encontramos uma melhor estimativa para o pico. Vamos fazer isso para os dados da Alemanha, pois como está num momento mais avançado da epidemia, podemos imaginar que estamos em um momento anterior ao pico e ver se poderíamos estimar o que aconteceria nos momentos posteriores. Ou seja, o exemplo a ser mostrado é uma suposição que estamos olhando a Alemanha num momento anterior do tempo, sem saber como seria no futuro. Mas vamos comparar com os dados que já sabemos.

### Fit numa sigmoid

In [ ]:
acum_al = df_conf_w['Germany'][df_conf_pd[df_conf_pd['Germany'] > 10].index[0]:].values
acum_al_treino = acum_al[:30]
acum_al_teste = acum_al[30:]

In [ ]:
# Aqui coloquei que o espaço de busca é pelo menos o q tivemos vezes, dois, pois vemos que não estamos no pico
boundaries = ([0., 0.05, acum_al_treino.max()*2],[90., 1.0, 100*acum_al_treino.max()])
xal, yal, erral, popt_sigal = sigm_predict(acum_al_treino, 30, boundaries)

In [ ]:
#plt.plot(np.linspace(0,len(acum_al),len(acum_al)), acum_al, 'y', label='Real')
plt.plot(np.linspace(0,len(acum_al_treino),len(acum_al_treino)), acum_al_treino, 'b', label='Passado')
plt.plot(np.linspace(len(acum_al_treino),len(acum_al_treino)+30, 30), acum_al_teste[:30], 'g', label='Futuro')
plt.plot(np.linspace(0, len(acum_al_treino)+30,len(acum_al_treino)+30), yal, 'r', label = 'Pred. Al')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Hummmm. Se distanciou muito, né? Veja que o fit foi feito buscando valores de máximo da função sigmoid duas ordens de grandeza maiores que o valor até então, o que dá um espaço muito amplo de busca. 

De qualquer forma, se observarmos como seria a curva no crescimento diário, ao invés do acumulado, vemos que há uma aproximação que faz mais sentido do que as projeções anteriores (Linear, polinomial e sarima). Isso é porque fizemos o fit numa função que sabemos que terá decaimento, como nas epidemias, enquanto que as demais seguem a tendência anterior até que hajam dados suficientes de sua mudança, o que na prática significa que o pico já passou.

In [ ]:
pd_al_real = df_conf_pd['Germany'][df_conf_pd[df_conf_pd['Germany'] > 10].index[0]:].values
pd_al_prev = np.diff(yal)

In [ ]:
plt.plot(pd_al_real, label='Real')
plt.plot(pd_al_prev, label='Prev Sigmoid')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

Que interessante. Os modelos anteriores seguiam a tendência observada anteriormente, logo, para pontos futuros, divergiam muito da realidade quando da mudança natural de tendência epidêmica. Já assumindo que sabemos que ela decairá, mesmo com muita diferença para o real, ela é menor do que os anteriores. Isso mostra como o conhecimento de domínio é importante.

Bom, nesse caso, tínhamos pego dados próximos do pico para projetar pontos posteriores, o fit da sigmoid logo projetou a mudança de tendência, nos ajudando no exemplo. Se pegássemos intervados bem anteriores ao pico para projeção, o fit da sigmoid começaria errar para bem menos. 

Isso se deve a uma instabilidade observada no modelo sigmoid quando temos variações de dados em sua fase de crescimento (intervalo de crescimento logo após o início exponencial). Ou seja, muitas possibilidades de curva sigmoid podem se aproximar aos dados de histórico em relação ao crescimento exponencial inicial e à aceleração observada, bastando mudar apenas o máximo de estabilização previsto. No exemplo demonstrado, colocamos um espaço muito grande de variação desse máximo (2x o valor máximo atual a 100x ele).

Para melhor estabeler esses máximos de estabilização, novamente é necessário conhecimento de domínio. Por exemplo, qual é o alcance do fenômeno? É o tamanho da população?

No caso de epidemias, sabemos que não há necessidade de atingir toda a população para haver imunização de rebanho. Então, podemos observar o compartamento de epidemias similares anteriores, ou ainda, casos de países em estágios epidêmicos mais avançados. Ambas as alternativas podem ajustar melhor o modelo à realidade, do que simplesmente olhar dados anteriores e projetar o futuro. Se olharmos exemplos representativos de números de casos de outros países, podemos ter aproximações melhores. Ainda, eles já podem considerar estatísticas embutidas da amostragem que ocorre em na quantidade de testes pelo tamanho da população.


Pra facilitar, e considerando que meu objetivo é somente didático, vamos fazer isso: olhar dados de países próximos como se estivessem mais avançados no tempo, em relação à Alemanha (obs. sabemos q eles começaram bem próximos na epidemia, poderia usar os dados asiáticos, mas eles são bem diferentes do resto do mundo por motivos que pode ser alvo de outra análise). No caso, é justo normalizar os dados pelo tamanho da população de cada país, uma vez que são bem diferentes, então vamos fazer o número de casos por 1 milhão de habitantes. 

In [ ]:
eixo_dias = np.linspace(0,df_conf_w.shape[0], len(df_conf_w))
plt.plot(eixo_dias, df_conf_w['Italy']*1000000/pop[pop['local'] == 'Italy']['pop'].values, label = 'Italy')
plt.plot(eixo_dias, df_conf_w['France']*1000000/pop[pop['local'] == 'France']['pop'].values, label = 'France')
plt.plot(eixo_dias, df_conf_w['Switzerland']*1000000/pop[pop['local'] == 'Switzerland']['pop'].values, label = 'Spain')
plt.plot(eixo_dias, df_conf_w['Norway']*1000000/pop[pop['local'] == 'Norway']['pop'].values, label = 'Norway')
plt.xlabel("Dia")
plt.ylabel("Casos Acumulados")
plt.legend();

Bem, vemos valores bem diferentes. Isso tem muito a ver com fatores de cada país, como medidas de quarentena, amplitude da testagem, etc. Como vamos saber se a Alemanha vai desempenhar melhor ou pior que eles? Não dá, mas, estatisticamente, usar um intervalo mais factível para busca das projeções, pode nos dar uma ideia melhor de limites a estimar. Então vamos fazer uma projeção pra Alemanha considerando como limites o maior e o menor valores observados.

In [ ]:
#Dados por 1M, a partir de >10 casos
acum_al_1m = df_conf_w['Germany'][df_conf_pd[df_conf_pd['Germany'] > 10].index[0]:]*1000000/pop[pop['local'] == 'Germany']['pop'].values
acum_al_treino_1m = acum_al_1m[:30]
acum_al_teste_1m = acum_al_1m[30:]

In [ ]:
# Aqui o limite máximo será algum valor entre próximo de 1500 (Norway)
boundaries = ([0., 0.05, 1500],[90., 1.0, 2000])
xal, yal_1m_min, erral, popt_sigal = sigm_predict(acum_al_treino_1m, 30, boundaries)
print(popt_sigal[0])
# Um valor próximo do maior da Itália 3500
boundaries = ([0., 0.05, 3500],[90., 1.0, 4000])
xal, yal_1m_max, erral, popt_sigal = sigm_predict(acum_al_treino_1m, 30, boundaries)
print(popt_sigal[0])

In [ ]:
#plt.plot(np.linspace(0,len(acum_al_1m),len(acum_al_1m)), acum_al_1m, 'y', label='Real')
plt.plot(np.linspace(0,len(acum_al_treino_1m),len(acum_al_treino_1m)), acum_al_treino_1m, 'b', label='Passado')
plt.plot(np.linspace(len(acum_al_treino_1m),len(acum_al_treino_1m)+30, 30), acum_al_teste_1m[:30], 'g', label='Futuro')
plt.plot(np.linspace(0, len(acum_al_treino_1m)+30,len(acum_al_treino_1m)+30), yal_1m_min, 'r', label = 'Pred. Min')
plt.plot(np.linspace(0, len(acum_al_treino_1m)+30,len(acum_al_treino_1m)+30), yal_1m_max, 'r', label = 'Pred. Max')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend();

In [ ]:
fator = 1000000/pop[pop['local'] == 'Germany']['pop'].values[0]

In [ ]:
# Valores calculados por 1M, agora vontando aos absolutos
pd_al_prev_min = np.diff(yal_1m_min/fator)
pd_al_prev_max = np.diff(yal_1m_max/fator)

In [ ]:
plt.plot(pd_al_real, label='Real')
plt.plot(pd_al_prev_min, 'r', label='Prev Min')
plt.plot(pd_al_prev_max, 'r', label='Prev Max')
plt.xlabel("Dia")
plt.ylabel("Casos por dia")
plt.legend();

Bom, os intervalos calculados projetam valores de casos por dia bem diferentes. Nos picos, é um pouco mais que o dobro quando usamos os comportamentos de Noruega e Itália como exemplo. Então o que mostramos é como seria a tendência da Alemanha se ela se comportasse como um país ou outro. No caso, vemos que ela está um pouco pior do que o primeiro, mas muito melhor que o último. O conhecimento de domínio poderia ajudar a estimar melhor, se considerarmos, por exemplo, que a Alemanha se assemelha muito mais em características com a Noruega.

De qualquer forma, mesmo pegando o caso extremo da Itália como referência, **podemos projetar um pico de pior caso, que os modelos de regressão linear, polinimial e Sarima não seriam capazes de fazer.**

Se o objetivo é descobrir onde está o pico, temos um intervalo bem melhor para planejamento e tomada de decisão. No caso, a diferença entre os projetados é de 7 dias.



### Facebook's Prophet

Um modelo muito interessante de previsão de séries temporais, é o modelo Prophet desenvolvido pelo Facebook (https://facebook.github.io/prophet/docs/installation.html#python). Ele é muito prático por já considerar a composição de diferentes possibilidades de tendências (como a logística) e diferentes componentes sazonais, como diária, mensal, de feriados etc. Ou seja, é um modelo que permite a introdução mais fácil de conhecimento de domínio.

Mas ele precisa da definição dos limites da curva, assim como mostrado anteriormente. Vamos usar o valor máximo no meio entre Noruega e Itália (2500), novamente, assumindo conhecimento de domínio. 

O objetivo dessa experiência aqui é mostrar que a mesma ideia do conhecimento de domínio é válida para qualquer modelo de previsão de séries temporais, bastando fazer a correta parametrização. Nesse caso, vemos inclusive que o modelo tenta capturar a sazonalidade semanal dos dados, além da tendência. Coisa que a projeção anterior não faz. O resultado da sazonalidade não é tão bom com o passar do tempo de previsão, talvez o modelo possa ser melhor parametrizado, mas acertar 100% nem Mãe Diná!

In [ ]:
# Precisa estar num dataframe com data
df_al = df_conf_w['Germany'][df_conf_pd[df_conf_pd['Germany'] > 10].index[0]:].copy()
df_al = df_al.reset_index()
df_al.rename(columns={'Date':'ds', 'Germany':'y'}, inplace=True)
df_al_treino = df_al[:30]
df_al_teste = df_al[30:]

In [ ]:
from fbprophet import Prophet

prophet = Prophet(growth='logistic',
                  changepoint_range=0.95,
                  yearly_seasonality=False,
                  weekly_seasonality=True,
                  daily_seasonality=False,
                  #seasonality_prior_scale=10,
                  changepoint_prior_scale=.5)#.01
#Vamos estabelecer os limites como anteriormente
cap = 2000/fator#10*df_al_treino['y'].max()
floor = 0#2*df_al_treino['y'].max()
df_al_treino['cap'] = cap
df_al_treino['floor'] = floor
proffit = prophet.fit(df_al_treino)

#fitted = m_sarima.predict_in_sample(start=1, end=len(casos_al_treino))
futuro = prophet.make_future_dataframe(periods=30, freq='D')
futuro['cap'] = cap
futuro['floor'] = floor
casos_al_prev_prof = prophet.predict(futuro)
fig = prophet.plot(casos_al_prev_prof)
#a = add_changepoints_to_plot(fig.gca(), prophet, forecast)
plt.plot(futuro['ds'][30:], df_al_teste['y'][:30], 'g', label = 'Futuro')
plt.show()
#fig2 = prophet.plot_components(forecast)
#plt.show()

In [ ]:
#Por dia
plt.figure(figsize=(10,4))
#plt.plot(np.linspace(0,len(casos_al), len(casos_al)), casos_al, 'b', label='Real')
plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), casos_al_treino, 'b', label='Passado')
plt.plot(np.linspace(len(casos_al_treino), len(casos_al), len(casos_al)-len(casos_al_treino)), casos_al[len(casos_al_treino):], 'g', label='Futuro')
plt.plot(np.linspace(0,len(casos_al_treino)+30, len(casos_al_treino)+30), casos_al_prev_prof['yhat'].diff().values, 'r', label='Prophet')
#plt.plot(np.linspace(0,len(casos_al_treino), len(casos_al_treino)), fitted, 'c')
plt.xlabel("Dia")
plt.ylabel("Casos")
plt.legend()

# Conclusões

Prever o futuro próximo é viável!

Prever o futuro mais distante é difícil!

Observar tendências e sazonalidades podem nos fazer enxergar certos padrões nos dados. Com esses padrões, podemos estimar melhor as possibilidades futuras.

Conhecimento de domínio é importante para estimar melhor.

Mas isso não é futurologia! Sempre interprete os resultados com cautela e considere as condições de contorno.

# A Fazer



*   Avaliar modelos com métricas de erro
*   Projetar curvas do Brasil
*   Incluir modelos LSTM e DeepAR

